In [2]:
input = ["(dw-p)x3", "(dw-p)x3", "(dw-p)x3", "(dw-p)"]
input[0]

'dw-p'

In [1]:
import torch

/home/brotherhoon88/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LayerNorm(nn.Module):
    def __init__(self,
                 normalized_shape,
                 eps=1e-6,
                 data_format="channels_first"):
        super(LayerNorm, self).__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape)) # dim
        self.bias = nn.Parameter(torch.zeros(normalized_shape)) # dim
        self.eps = eps # 분모에 더해지는 작은 값
        self.data_format = data_format
        if self.data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError
        self.normalized_shape = (normalized_shape, )
        
    def forward(self, x:torch.Tensor):
        if self.data_format == "channels_last":
            return F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        elif self.data_format == "channels_first":
            u = x.mean(dim=1, keepdim=True)
            s = (x-u).pow(2).mean(1, keepdim=True)
            x = (x-u)/torch.sqrt(s + self.eps)
            x = self.weight[:, None, None]*x + self.bias[:, None, None]
            return x
        
LayerNorm(96,data_format="channels_first")(torch.Tensor(64, 96, 58, 58)).shape

torch.Size([64, 96, 58, 58])

In [5]:
import torch
import torch.nn as nn

input = torch.Tensor(64,96,7,7)
avgpool = nn.AdaptiveAvgPool2d(output_size=(16,16))
avgpool(input).shape

16*16

256

In [8]:
x = torch.Tensor(1,1,2,2)
print(x)
y = x*10
print(y)
z = x + y
print(z ==  x+y)

tensor([[[[6.6728e-35, 0.0000e+00],
          [2.6768e+03, 4.5741e-41]]]])
tensor([[[[6.6728e-34, 0.0000e+00],
          [2.6768e+04, 4.5741e-40]]]])
tensor([[[[True, True],
          [True, True]]]])


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

conv0 = nn.Conv2d(
    in_channels=96,
    out_channels=96,
    kernel_size=7,
    stride=1,
    padding=int(7 // 2),
    groups=1,
    bias=True,
)
print(conv0.weight.shape, conv0.bias.shape) # torch.Size([96, 1, 7, 7]) torch.Size([96]), torch.Size([96, 96, 7, 7]) torch.Size([96])

input = torch.randn(32, 96, 52, 52)
input = input.reshape(1, -1, 52, 52)
attn_weight = torch.Tensor(32 * 96, 1, 7, 7)
attn_bias = torch.Tensor(32*96)

out = F.conv2d(
    input, weight=attn_weight, bias=attn_bias, stride=1, padding=int(7 // 2), groups=32 * 96
)

out.shape

/home/brotherhoon88/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.Size([96, 96, 7, 7]) torch.Size([96])


torch.Size([1, 3072, 52, 52])

In [ ]:
import torch
import torch.nn.functional as F

# 예시 입력 데이터
batch_size = 10
input_channels = 3
input_height = 32
input_width = 32

# 커널 크기와 스트라이드 설정
kernel_size = 3
stride = 1
padding = 1

# Depthwise Convolution 수행
input_data = torch.randn(batch_size, input_channels, input_height, input_width)
depthwise_weight = torch.randn(input_channels, 1, kernel_size, kernel_size)  # depthwise convolution의 가중치
depthwise_bias = torch.randn(input_channels)  # depthwise convolution의 bias

# depthwise convolution 수행
output = F.conv2d(input_data, depthwise_weight, bias=depthwise_bias, stride=stride, padding=padding, groups=input_channels)

# 출력 크기 확인
print(output.shape)


In [ ]:
from torchsummary import summary
from fvcore.nn import FlopCountAnalysis, flop_count_table


def profile_model(model:nn.Module):
    M=model # your model
    summary(M, (3, 224, 224), batch_size=256, device="cpu") #모델 서머리
    flops = FlopCountAnalysis(M, torch.Tensor(1,3,224,224))
    print(flop_count_table(flops)) #프로파일링
    formatted_number = "{:.2f}G".format(flops.total() / 1e9)
    print(formatted_number) # 플롭스